# Análise Setorial

<p style="font-size:16px; padding-left: 30px;"><strong style="color: #3583DE;">dbo.setorial_f_pib</strong>

<p style="font-size:14px; padding-left: 40px;"><strong style="color: #3583DE;">Saída:</strong> [iddata], [pib].</p>

<p style="font-size:16px; padding-left: 30px;"><strong style="color: #3583DE;">dbo.setorial_ipca</strong>

<p style="font-size:14px; line-height: 50%; padding-left: 40px;"><strong style="color: #3583DE;">Saída:</strong> [iddata], [meta], [previsto], [realizado].</p>

## Libs

In [2]:
import pandas as pd 
from datetime import date, datetime

from Config.utils import *
from Config.connect import *
from Config.utils import *
from Extract.get_IPCA import get_ipca 
from Extract.get_PIB import get_pib 
from Extract.get_PNAD import get_pnad
from Extract.get_SELIC import get_selic
from Load.insert_SQL import insert

## Variáveis

In [3]:
Atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
Ano = date.today().year

# Insira o local para salvamento dos arquivos csv que serão utilizados para o insert na base de dados.
bcp_path = "D:/"

## Execute

In [4]:
print('')
print(f'{Atual}: Buscando dados...')
print('')

ipca_meta, ipca_previsto, ipca_realizado = get_ipca(2024)
pib = get_pib()
pnad = get_pnad()
selic = get_selic()


20/03/2024 01:24:07: Buscando dados...



## Saída

### Insere os dados na base SQL Server

In [ ]:
#Insira em saida_dict as tabelas que serão populadas na base ao lado dos respectivos DataFrames. 
saida_dict = {
    ipca_meta: 'setorial_f_ipca_meta',
    ipca_previsto: 'setorial_f_ipca_previsto',
    ipca_realizado: 'setorial_f_ipca_realizado',
    pib: 'setorial_f_pib',
    pnad: 'setorial_f_desemprego',
    selic: 'setorial_f_selic'
}

for df, tabela in saida_dict.items():
    insert(df = df, schema = 'dbo', tabela = tabela, bcp_path = bcp_path)

### Atualiza Power BI DataSet

In [ ]:
update_url = f'https://api.powerbi.com/v1.0/myorg/groups/{ws_setorial}/datasets/{ds_setorial}/refreshes'
response = requests.post(update_url, headers=PBIheaders)

if response.status_code == 202:
    MsgPBI = 'DS Análise Setorial - Solicitação de atualização enviada com sucesso.'
else:
    MsgPBI = 'DS Análise Setorial - Ocorreu um erro ao enviar a solicitação de atualização.'
    MsgPBI = 'DS Análise Setorial - Código de status:', response.status_code
    MsgPBI = 'DS Análise Setorial - Resposta:', response.json()

print('')
print(MsgPBI)
print('')

input("Pressione Enter para sair...")